In [5]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KDTree
from scipy.spatial.distance import euclidean

class RGCLI:
    """ Algoritmo de construcción de grafos RGCLI basado en el articulo:
        'RGCLI: Robust Graph that Considers Labeled Instances for Semi-
        Supervised Learning' de los autores:
        - Lilian Berton, Alan Valejo, Thiago de Paulo Faleiros, Jorge Valverde-Rebaza
        Alnea de Andrade Lopes
    """
    def __init__(self, datos_e, datos_se, etiquetas, ke, ki):
        """ Inicializa el algoritmo RGCLI
        
        Args:
        - datos_e: np.array
            datos etiquetados
        - datos_se: np.array
            datos sin etiquetar
        - etiquetas: np.array
            Todas las etiquetas
        - Ke: int
            Número de vecinos más cercanos
        - Ki: int
            Número de vecinos más cercanos para el RGCLI
        """
        self.nodos = np.concatenate((datos_e, datos_se), axis=0)
        self.nodos_etiquetados = np.array(range(len(datos_e)))
        self.nodos_sin_etiquetar = np.array(range(len(datos_e), len(datos_e) + len(datos_se)))

        self.etiquetas_etiquetados = etiquetas[:len(datos_e)]
        self.etiquetas_modificadas = np.concatenate((self.etiquetas_etiquetados, np.full(len(datos_se), -1)))

        self.ke = ke
        self.ki = ki
        self.V = list(range(len(self.nodos)))
        self.E = []
        self.W = {}
        self.kdtree = KDTree(self.nodos)
        self.l_kdtree = KDTree(self.nodos[self.nodos_etiquetados, :])
        self.kNN = {}
        self.F = {}
        self.L = {}
        self.grafo_knn = {}
        self.grafoFinal = {}

    def search_knn(self):
        """
        Construye el grafo de vecinos mas cercanos.

        Para cada nodo en self.V, este método:
        - Encuentra todos los vecinos del nodo en el espacio de características, 
            utilizando un kdtree para la búsqueda eficiente.
        - Almacena los primeros `ke` vecinos en `self.kNN` y `self.grafo_knn`.
        - Encuentra los vecinos etiquetados del nodo y almacena el más cercano en `self.L`.
        - Almacena el vecino `ke`-ésimo más lejano en `self.F`.

        Returns:
            dict: Un diccionario que mapea cada nodo en self.V a sus `ke` vecinos más cercanos.
        """
        for v in self.V:
            all_neighbors = self.kdtree.query([self.nodos[v]], k=len(self.nodos), return_distance=False)[0]
            self.kNN[v] = all_neighbors[1:self.ke + 1].tolist()
            self.grafo_knn[v] = self.kNN[v]

            labeled_neighbors = self.l_kdtree.query([self.nodos[v]], k=2, return_distance=False)[0]
            self.L[v] = self.nodos_etiquetados[labeled_neighbors[labeled_neighbors != v][0]]
            self.F[v] = all_neighbors[-self.ke]
        return self.kNN

    def search_rgcli(self):
        """
        Construye un grafo final con ayuda de los datos etiquetados.

        Para cada nodo en self.V, este método realiza lo siguiente:
        - Calcula una medida de distancia, epsilon, para cada vecino del nodo.
        - Selecciona los `ki` vecinos con la menor medida epsilon y los añade a `self.E`.
        - Asigna un peso de 1 a cada uno de estos vecinos en `self.W`.
        - Añade estos vecinos al grafo final `self.grafoFinal`.

        El grafo final es un grafo no dirigido, cada nodo está conectado a sus `ki` vecinos.
        """
        for vi in self.V:
            epsilon = dict()
            for vj in self.kNN[vi]:
                if euclidean(self.nodos[vi], self.nodos[vj]) <= euclidean(self.nodos[vj], self.nodos[self.F[vj]]):
                    e = (vi, vj)
                    epsilon[e] = euclidean(self.nodos[vi], self.nodos[vj]) + euclidean(self.nodos[vj], self.nodos[self.L[vj]])
            E_prime = sorted(epsilon, key=epsilon.get)[:self.ki]
            self.E.extend(E_prime)
            for e in E_prime:
                print(e)
                self.W[e] = 1
                # grafo no dirigido y no redundante 
                
                if e[0] not in self.grafoFinal:
                    self.grafoFinal[e[0]] = []
                if e[1] not in self.grafoFinal[e[0]]:
                    self.grafoFinal[e[0]].append(e[1])
                    
                if e[1] not in self.grafoFinal:
                    self.grafoFinal[e[1]] = []
                if e[0] not in self.grafoFinal[e[1]]:
                    self.grafoFinal[e[1]].append(e[0])
                
                
    def run(self):
        """
        Metodo que ejecuta los pasos del algoritmo RGCLI
        
        Returns:
            dict, dict: Un diccionario para el grafo del primer paso y otro para el grafo final.   
        """
        self.search_knn()
        self.search_rgcli()
        return self.grafo_knn, self.grafoFinal

#



In [6]:
from matplotlib import pyplot as plt
import networkx as nx
from collections import deque, defaultdict
import numpy as np
from scipy.spatial import distance_matrix
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from copy import deepcopy


class Gbili:
    """ Algoritmo de construccion de grafos GBILI basado en el artículo:
    'Graph construction based on labeled instances for
    Semi-Supervised Learning' de los autores: 
    Lilian Berton y Alneu de Andrade Lopes
    """

    def __init__(self, datos_se, datos_e, etiquetas, K):
        """
        Constructor de la clase.

        Args:
        - datos_se: numpy.ndarray
            Datos sin etiquetar.
        - datos_e: numpy.ndarray
            Datos etiquetados.
        - etiquetas: numpy.ndarray
            Todas las etiquetas.
        - K: int
            Número de vecinos más cercanos a considerar.

        Inicializa los atributos:
        - nodos: numpy.ndarray
            Todos los nodos (datos reales), tanto etiquetados como sin etiquetar.
        - nodos_etiquetados: numpy.ndarray
            Índices de los nodos etiquetados.
        - nodos_sin_etiquetar: numpy.ndarray
            Índices de los nodos sin etiquetar.
        - K: int
            Número de vecinos más cercanos a considerar.
        - matriz_distancias: numpy.ndarray
            Matriz de distancias entre los nodos.
        - grafo: dict
            Grafo que representa las conexiones entre los nodos.
        - etiquetas_etiquetados: numpy.ndarray
            Etiquetas correspondientes a los nodos etiquetados.
        """

        self.nodos = np.concatenate((datos_e, datos_se), axis=0)
        self.nodos_etiquetados = np.array(range(len(datos_e)))
        self.nodos_sin_etiquetar = np.array(range(len(datos_e), len(datos_e) + len(datos_se)))
        self.K = K
        self.matriz_distancias = distance_matrix(self.nodos, self.nodos)
        self.grafo = {}

        
        self.etiquetas_etiquetados = etiquetas[:len(datos_e)]
        # TODO: borrar
        self.etiquetas_modificadas = np.concatenate((self.etiquetas_etiquetados, np.full(len(datos_se), -1)))
    def construir_grafo(self):
        """
        Indica el proceso de construcción del grafo GBILI.
        Se realizan los siguientes pasos:
        1. Encuentra los k vecinos más cercanos de cada vértice.
        2. Encuentra los pares mutuos de vecinos más cercanos.
        3. Conecta los vértices con la mínima distancia.
        4. Encuentra los componentes del grafo.
        5. Conecta los componentes del grafo basándose en la presencia de etiquetas.

        Returns:
            dict: grafo resultante de la construcción.
        """
        # Configurar la figura y los ejes
        # fig, axs = plt.subplots(2, 2, figsize=(12, 6))  # 2 fila, 2 columnas
        # colores_map = np.array(self.etiquetas_modificadas)

        lista_knn = self.encuentra_knn()
        # self.dibujar_grafo(lista_knn, colores_map, axs[0, 0], "K-NN")

        lista_mknn = self.encuentra_mknn(lista_knn)
        # self.dibujar_grafo(lista_mknn, colores_map, axs[0, 1], "Mutual K-NN")

        self.conectar_minima_distancia(lista_mknn)
        grafo_1 = deepcopy(self.grafo)

        # Componente = subgrafo
        componentes = self.encontrar_componentes()   
        # print("Componentes antes de conectar: ", componentes) 
        self.conectar_componentes(componentes, lista_knn)
        # print("GRAFO CON COMPONENTES CONECTADOS: ", self.grafo)
        # print()
        grafo_2 = deepcopy(self.grafo)
        # print("Componentes despues de conectar: ", self.encontrar_componentes())

        # self.dibujar_grafo(grafo_1, colores_map, axs[1, 0], "Grafo antes de conectar componentes")
        # self.dibujar_grafo(grafo_2, colores_map, axs[1, 1], "Grafo después de conectar componentes")
        # Crear una leyenda
        custom_lines = [Line2D([0], [0], color='yellow', lw=3),
                        Line2D([0], [0], color='blue', lw=3),
                        Line2D([0], [0], color='green', lw=3),
                        Line2D([0], [0], color='grey', lw=3)]

        # fig.legend(custom_lines, ['0', '1', '2', 'Desconocido'])
        # plt.show()
        return self.grafo

    def encuentra_knn(self):
        """ Encuentra los k vecinos más cercanos de cada vértice.
        <<knn>>: `k-nearest neighbors`

        Returns:
            dict: contiene los k vecinos más cercanos de cada vértice.
        """
        n = len(self.nodos)
        lista_knn = {}
        for i in range(n):
            # Guarda una tupla con la distancia y el indice del vertice
            distancias = [(self.matriz_distancias[i][j], j) for j in range(n) if i != j]
            distancias.sort()
            # Seleccionar los K vecinos más cercanos
            lista_knn[i] = [ind_v for _, ind_v in distancias[:self.K]]
        return lista_knn

    def encuentra_mknn(self, lista_knn):
        """ Para los vértices en la lista de k-NN
        encuentra los pares mutuos de vecinos más cercanos.
        Mutuo k-NN: Si i es vecino más cercano de j y 
                    j es vecino más cercano de i.
        Args:
            lista_knn (dict): contiene los k vecinos más cercanos de cada vértice.

        Returns:
            dict: contiene los pares mutuos de vecinos más cercanos.
        """
        lista_mknn = {}
        for i in lista_knn:
            lista_mknn[i] = []
            for j in lista_knn[i]:
                if i in lista_knn[j]:
                    lista_mknn[i].append(j)
        return lista_mknn

    def conectar_minima_distancia(self, lista_mknn):
        """ Conecta los vértices con la mínima distancia.
        Almacena esos enlaces en el diccionario que representa el grafo.
        El grafo es no dirigido, por lo que se conectan ambos nodos.

        Args:
            lista_mknn (dict): contiene los pares mutuos de vecinos más cercanos.
        """
        for i in lista_mknn:
            # Si el nodo no tiene vecinos mutuos, se agregan como elementos solitarios
            if not lista_mknn[i]:
                self.grafo[i] = []
                continue
            for j in lista_mknn[i]:
                min_distancia = float('inf')
                min_enlace = None
                for l in self.nodos_etiquetados:
                    d = self.matriz_distancias[i][j] + self.matriz_distancias[j][l]
                    if d < min_distancia:
                        min_distancia = d
                        min_enlace = (i, j)
            if min_enlace:
                if min_enlace[0] not in self.grafo:
                    self.grafo[min_enlace[0]] = []
                if min_enlace[1] not in self.grafo:
                    self.grafo[min_enlace[1]] = []
                if min_enlace[1] not in self.grafo[min_enlace[0]]:
                    self.grafo[min_enlace[0]].append(min_enlace[1])
                if min_enlace[0] not in self.grafo[min_enlace[1]]:
                    self.grafo[min_enlace[1]].append(min_enlace[0])

    def encontrar_componentes(self):
        """ Encuentra los componentes del grafo.
        Utiliza BFS (Breadth First Search) o busqueda en anchura 
        para encontrar los componentes.

        Returns:
            dict: contiene los componentes del grafo.
        """
        visitados = {}
        componentes = {}
        id_comp = 0

        def bfs(v):
            """Realiza una búsqueda en anchura a partir de un vértice dado.

            Esta función implementa el algoritmo de búsqueda en anchura
            (BFS por sus siglas en inglés) en un grafo. 
            Comienza la búsqueda desde el vértice 'v' y visita todos los 
            vértices alcanzables desde 'v'.

            Args:
                v (int): El vértice de inicio para la búsqueda en anchura.
            """
            cola = deque([v])
            visitados[v] = True
            componentes[v] = id_comp
            while cola:
                actual = cola.popleft()
                for vecino in self.grafo.get(actual, []):
                    if not visitados.get(vecino, False):
                        visitados[vecino] = True
                        componentes[vecino] = id_comp
                        cola.append(vecino)

        for v in range(len(self.nodos)):
            if v not in visitados:
                bfs(v)
                id_comp += 1
        return componentes
 
    def conectar_componentes(self, comp, lista_knn):
        """
        Conecta los componentes del grafo basándose en la presencia de etiquetas.

        Args:
            comp (dict): contiene los componentes del grafo con cada nodo mapeado a su componente.
            lista_knn (dict): contiene los k vecinos más cercanos de cada vértice.
        """
        # Invertir el diccionario de componentes para agrupar nodos por componente
        comp_a_nodos = defaultdict(list)
        for nodo, componente in comp.items():
            comp_a_nodos[componente].append(nodo)

        # Determinar si cada componente tiene nodos etiquetados
        comp_etiquetados = {}
        for componente, nodos in comp_a_nodos.items():
            comp_etiquetados[componente] = any(nodo in self.nodos_etiquetados for nodo in nodos)

        # Conectar componentes según las condiciones dadas
        for v in range(len(self.nodos)):
            componente_v = comp[v]
            # Verificar si la componente de v no tiene nodos etiquetados
            if not comp_etiquetados[componente_v]:
                for vk in lista_knn[v]:
                    componente_vk = comp[vk]
                    # Verificar si la componente de vk tiene nodos etiquetados
                    if comp_etiquetados[componente_vk]:

                        if vk not in self.grafo[v]:
                            self.grafo[v].append(vk)
                        if v not in self.grafo[vk]:
                            self.grafo[vk].append(v)


In [7]:
"""Este módulo contiene la implementación del algoritmo Local Global Consistency (LGC)
para la clasificación de nodos en grafos.

@Autor:     Mario Sanz Pérez
@Fecha:     08/05/2024
@Versión:   1.0
@Nombre:    localglobalconsistency.py
"""""

from copy import deepcopy
import numpy as np

class LGC:
    """ Algoritmo de inferencia de etiquetas en grafos basado en el algoritmo Local Global Consistency (LGC).
    Articulo original: Zhou, D., Bousquet, O., Lal, T. N., Weston, J., & Schölkopf, B. (2004). 
    Learning with local and global consistency.
    """
    def __init__(self, grafo, nodos, etiquetas_etiquetados, alpha=0.5, tol=1e-6, max_iter=1000):
            """Inicializa una instancia de la clase LocalGlobalConsistency.

            Args:
                grafo (tipo): El grafo utilizado para el algoritmo.
                nodos (tipo): Los nodos del grafo.
                etiquetas_etiquetados (tipo): Las etiquetas de los nodos etiquetados.
                alpha (float): El parámetro de suavizado entre la matriz de afinidad y las etiquetas.
                tol (float): La tolerancia para la convergencia del algoritmo.
                max_iter (int): El número máximo de iteraciones.
            """
            self.grafo = grafo
            self.nodos = nodos
            self.etiquetas_etiquetados = etiquetas_etiquetados
            self.n_categorias = len(np.unique(self.etiquetas_etiquetados))
            self.Y = self.inicializar_Y()
            self.matriz_afinidad = self.construir_matriz_afinidad()
            self.Y = self.inicializar_Y()
            self.alpha = alpha
            self.tol = tol
            self.max_iter = max_iter

    def inicializar_Y(self):
        """ Inicializa la matriz de etiquetas Y (mascara). 
        Se inicializa con ceros y se asigna un 1 en la columna correspondiente a la etiqueta.
        Los nodos no etiquetados tienen todas las columnas a cero."""
        
        Y = np.zeros((len(self.nodos), self.n_categorias))
        for i, label in enumerate(self.etiquetas_etiquetados):
            Y[i, label] = 1
        return Y    
    
    def construir_matriz_afinidad(self):
        """ Construye una matriz de distancias simplificada del grafo.
        Se asume que la distancia entre nodos vecinos es 1 y el resto de distancias es 0.
        Returns:
            np.array: La matriz de distancias del grafo.
        """
        W = np.zeros((len(self.grafo), len(self.grafo)))
        for nodo, vecinos in self.grafo.items():
            for vecino in vecinos:
                W[nodo][vecino] = 1
        return W

    def inferir_etiquetas(self):
        """ Proceso de inferencia de etiquetas en el grafo.
        1. Construir la matriz de afinidad W (ponderada a 1 y 0).
        2. Normalizar la matriz de afinidad S.
        3. Iterar F hasta convergencia.
        4. Predecir las etiquetas de los nodos no etiquetados.
        
        Returns:
            np.array: Las etiquetas inferidas de los nodos no etiquetados.
        """

        S = self.normalizar_afinidad(self.matriz_afinidad)
        F_final = self.iterar_F(S, alpha=self.alpha, tol=self.tol, max_iter=self.max_iter)
        return self.predecir_etiquetas(F_final)

    def normalizar_afinidad(self, W, epsilon=1e-4):
        """ Normaliza la matriz de afinidad W.
            Args:
            W (np.array): La matriz de afinidad W.
            epsilon (float): Un valor cercano a 0 para evitar indeterminaciones.
            
            Returns:
            np.array: La matriz de afinidad normalizada S.
        """
        D = np.diag(W.sum(axis=1) + epsilon)
        D_inversa = np.diag(1 / np.sqrt(D.diagonal()))
        S = D_inversa @ W @ D_inversa
        
        return S

    def iterar_F(self, S, alpha=0.5, tol=1e-6, max_iter=10000):
        """ Itera la matriz de etiquetas F hasta convergencia.
        Args:
            S (np.array): La matriz de afinidad normalizada.
            alpha (float): El parámetro de suavizado entre la matriz de afinidad y las etiquetas.
            tol (float): La tolerancia para la convergencia del algoritmo.
            max_iter (int): El número máximo de iteraciones.
        Returns:
            np.array: La matriz de etiquetas F.
        """
        F = deepcopy(self.Y)
        for _ in range(max_iter):
            F_next = alpha * S @ F + (1 - alpha) * self.Y
            if np.linalg.norm(F_next - F) < tol:
                print("Convergencia alcanzada.")
                break
            F = F_next
        print("Iteraciones: ", _)
        
        return F

    def predecir_etiquetas(self, F):
        """ Predice las etiquetas de los nodos no etiquetados.
        
        Args:
            F (np.array): La matriz de etiquetas F.
            
        Returns:
            np.array: Las etiquetas inferidas de los nodos no etiquetados.
        """
        return np.argmax(F, axis=1)


In [8]:
# Ejemplo de uso con el dataset iris
from sklearn.datasets import load_iris, load_breast_cancer, load_digits, fetch_covtype

#data = load_iris()
# data = load_breast_cancer()
data = load_digits()
X = data.data
y = data.target

# Dividir en 80% para test (unlabeled) y 20% para entrenamiento (labeled)
L, U, L_, U_ = train_test_split(X, y, test_size=0.7, stratify=y)
todas_etiquetas = np.concatenate((L_, U_))

ke = 5
ki = 3

rgcli = RGCLI(L, U, todas_etiquetas, ke, ki)
grafo_knn, grafoFinal = rgcli.run()
print("******RGCLI******")

# print("Grafo KNN Construido:")
print(grafo_knn)
# print("Grafo Final Construido:")
#ordear grafo poniendo las keys en orden ascendente (no los valores)
sorted_keys = sorted(grafoFinal.keys())
grafoFinal = {key: grafoFinal[key] for key in sorted_keys}
# print(grafoFinal)
inferecia = LGC(grafoFinal, rgcli.nodos, rgcli.etiquetas_etiquetados, alpha=0.99, tol=1e-3, max_iter=1000)
predicciones = inferecia.inferir_etiquetas()

predicciones[len(L):]
etiquetas_reales = todas_etiquetas[len(L):]
accuracy = np.mean(predicciones[len(L):] == etiquetas_reales)
# print("Predicciones: ", predicciones[len(L):])
# print("Etiquetas reales: ", etiquetas_reales)
print(f"Accuracy: {accuracy}")

print("******GBILI******")
gbili = Gbili(U, L, todas_etiquetas, ke)
grafoGbili = gbili.construir_grafo()
inferecia = LGC(grafoGbili, gbili.nodos, gbili.etiquetas_etiquetados, alpha=0.99, tol=0.1, max_iter=10000)
predicciones = inferecia.inferir_etiquetas()

predicciones[len(L):]
etiquetas_reales = todas_etiquetas[len(L):]
accuracy = np.mean(predicciones[len(L):] == etiquetas_reales)
# print("Predicciones: ", predicciones[len(L):])
# print("Etiquetas reales: ", etiquetas_reales)
print(f"Accuracy: {accuracy}")

# dibujar los grafos con networkx
import networkx as nx
import matplotlib.pyplot as plt

# G = nx.Graph(grafoFinal)
# pos = nx.spring_layout(G)
# nx.draw(G, pos, with_labels=True, font_weight='bold')
# plt.show()

(0, 227)
(0, 1467)
(0, 1491)
(1, 1476)
(1, 1711)
(1, 39)
(2, 316)
(2, 259)
(2, 1093)
(3, 1606)
(3, 773)
(3, 241)
(4, 1504)
(4, 646)
(4, 561)
(5, 1204)
(5, 1445)
(5, 1636)
(6, 1745)
(6, 453)
(6, 229)
(7, 1628)
(7, 1379)
(7, 1645)
(8, 1403)
(8, 1628)
(8, 1379)
(9, 1467)
(9, 1653)
(9, 856)
(10, 649)
(10, 1033)
(10, 409)
(11, 813)
(11, 1678)
(11, 342)
(12, 1459)
(12, 94)
(12, 360)
(13, 154)
(13, 1550)
(13, 562)
(14, 1279)
(14, 798)
(14, 1020)
(15, 613)
(15, 1070)
(15, 1478)
(16, 252)
(16, 1545)
(16, 520)
(17, 193)
(17, 422)
(17, 1130)
(18, 1158)
(18, 545)
(18, 1085)
(19, 563)
(19, 772)
(19, 1462)
(20, 1214)
(20, 244)
(20, 751)
(21, 1205)
(21, 1685)
(21, 510)
(22, 276)
(22, 934)
(22, 723)
(23, 996)
(23, 1097)
(23, 1412)
(24, 1155)
(24, 990)
(24, 1193)
(25, 684)
(25, 219)
(25, 175)
(26, 280)
(26, 111)
(26, 869)
(27, 1653)
(27, 1467)
(27, 1307)
(28, 299)
(28, 1004)
(28, 394)
(29, 32)
(29, 642)
(29, 106)
(30, 408)
(30, 1233)
(30, 502)
(31, 1246)
(31, 711)
(31, 1539)
(32, 843)
(32, 367)
(32, 10

AttributeError: 'RGCLI' object has no attribute 'X'

In [ ]:
import numpy as np

# Rutas a los archivos
data_file = 'square.dat'
label_file = 'square.label'

# Leer el archivo de datos
X = np.loadtxt(data_file)

# Leer el archivo de etiquetas
y = np.loadtxt(label_file, dtype=int)

# Mostrar una parte de los datos para verificar
print("Datos (X):")
print(X[:5])  # Mostrar las primeras 5 filas de los datos

print("Etiquetas (y):")
print(y[:5])  # Mostrar las primeras 5 etiquetas

Datos (X):
[[0.5 2.5]
 [0.5 3. ]
 [0.5 3.5]
 [0.5 4. ]
 [0.5 4.5]]
Etiquetas (y):
[ 1 54]
